In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'PredictionCode/'
import os
import sys
sys.path.append(base_dir)
print(os.getcwd())
print(os.listdir())
os.chdir(base_dir)
print(os.getcwd())
print(os.listdir())

In [ ]:
import os
import torch
from torch.utils import data
from torchvision import transforms
from PIL import Image
import numpy as np
import re
import random
import shutil
import numpy as np
import cv2
import os
from os import listdir
from os.path import isfile, join


def transform_clip(video):
    t_ = transforms.Compose([transforms.ToPILImage(),
                                transforms.Resize((256, 256)),
                                transforms.ToTensor(),
                                ])
    new_clip = []
    for i in range(len(video)):
        frame = t_(np.array(video[i]))
        new_clip.append(frame)
    return torch.stack(new_clip)

def video_to_array():
        directories = [x[0] for x in os.walk('./hmdb51_org')]
        train_count = 0
        test_count = 0
        val_count = 0
        count = 0
        if os.path.exists('./PredData/train_data/'):
            shutil.rmtree('./PredData/train_data/')
        os.mkdir('./PredData/train_data/')

        if os.path.exists('./PredData/train_labels/'):
            shutil.rmtree('./PredData/train_labels/')
        os.mkdir('./PredData/train_labels/')

        if os.path.exists('./PredData/val_data/'):
            shutil.rmtree('./PredData/val_data/')
        os.mkdir('./PredData/val_data/')

        if os.path.exists('./PredData/val_labels/'):
            shutil.rmtree('./PredData/val_labels/')
        os.mkdir('./PredData/val_labels/')

        if os.path.exists('./PredData/test_data/'):
            shutil.rmtree('./PredData/test_data/')
        os.mkdir('./PredData/test_data/')

        if os.path.exists('./PredData/test_labels/'):
            shutil.rmtree('./PredData/test_labels/')
        os.mkdir('./PredData/test_labels/')

        
        label = 0
        labels = ['handstand', 'stand', 'smoke', 'somersault',
                     'clap', 'smile', 'sit', 'wave', 'laugh', 'cartwheel']
        for dir in directories:
            skip = True
            for l in labels:
                if dir.__contains__(l):
                    skip = False
            if skip:
                continue
            print(dir)
            videos = [f for f in listdir(dir) if isfile(join(dir, f))]
            for video in videos:
                arr = []
                cap = cv2.VideoCapture(join(dir, video))
                while cap.isOpened():
                    ret, frame = cap.read()
                    if frame is None:
                        break
                    arr.append(frame)
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                    if len(arr) == 70:
                        if count % 10 == 0:
                            torch.save(transform_clip(arr), './PredData/test_data/' + str(test_count) + '.pt')
                            torch.save(torch.tensor([label]), './PredData/test_labels/' + str(test_count) + '.pt')
                            test_count +=1
                        elif count % 10 == 1:
                            torch.save(transform_clip(arr), './PredData/val_data/' + str(val_count) + '.pt')
                            torch.save(torch.tensor([label]), './PredData/val_labels/' + str(val_count) + '.pt')
                            val_count +=1
                        else:
                            torch.save(transform_clip(arr), './PredData/train_data/' + str(train_count) + '.pt')
                            torch.save(torch.tensor([label]), './PredData/train_labels/' + str(train_count) + '.pt')
                            train_count +=1
                        arr = []
                        count +=1
                cap.release()
                cv2.destroyAllWindows()
            label += 1


video_to_array()
